In [2]:
import pandas as pd
import numpy as np
import ast
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
# get full text from extended tweet
def getExtTweet (row):
    if 'full_text' in row.keys():
        return row['full_text']
    else: return ""

In [4]:
# find the media type of media in tweet, if exists
def medTypeFind (row):
    for key in row.keys():
        if 'media' in key:
            return row['media'][0]['type']
        else: return np.nan

In [5]:
# get full text from retweeted status
def getFullText (row):  
    if 'extended_tweet' in row.keys():
        return row['extended_tweet']['full_text']
    else: return ""

In [6]:
def literaler (row):
    if type(row)==str:
        row = row.apply(ast.literal_eval)

In [7]:
# get original tweeter name if this tweet is a retweet
def getOg (row):
    if 'user' in row.keys():
        return row['user']['screen_name']
    else: return ""

In [37]:
#list of final column names
pklPath = "C:/Users/jacob/Desktop/Data Analytics 2022/2022 Ads/pickles/"
parqPath = "C:/Users/jacob/Desktop/Data Analytics 2022/2022 Ads/parquets/"
csvPath = "C:/Users/jacob/Desktop/Data Analytics 2022/2022 Ads/csvs/"
final_list = ['_id', 'screen_name', 'follower_count', 'location', 'Ad Name','created_at', 'mediaType', 'lang', 'source',
       'text', 'full_text','og_tweet_un', 'in_reply_to_screen_name', 'favorite_count','quote_count', 'reply_count', 'retweet_count', 'sentiment']

In [13]:
pklList = os.listdir('C:/Users/jacob/Desktop/Data Analytics 2022/2022 Ads/pickles')
csvNamelist = [sub.replace('.pkl','') for sub in pklList ]

In [44]:
all_df = pd.DataFrame(columns=final_list)
for pickle in pklList:
    df = pd.read_pickle(pklPath+pickle)
    # get formatting correct
    for i in range(0,df.columns.get_loc('favorite_count')):
        df.iloc[:,i] = df.iloc[:,i].astype(str)
    df = df.replace('nan', np.nan)
    df['created_at'] = df['created_at'].astype('datetime64[ns]')

    # get entities formatted as json, create mediatype column
    df['entities'] = df['entities'].apply(ast.literal_eval)
    df.insert(df.columns.get_loc('entities')+1, 'mediaType', np.nan)
    df['mediaType'] = df['entities'].apply(lambda cell : medTypeFind(cell))
    
    # format retweeted_status and extended_tweet columns as json, create full_text column
    df['retweeted_status'] = df['retweeted_status'].replace(np.nan, "{}")
    df['extended_tweet'] = df['extended_tweet'].replace(np.nan, "{}")
    df.insert(df.columns.get_loc('text')+1,'full_text',"")
    df['retweeted_status'] = df['retweeted_status'].apply(ast.literal_eval)
    df['extended_tweet'] = df['extended_tweet'].apply(ast.literal_eval)
    
    # extract full text wherever possible
    df['full_text'] = np.where(df['retweeted_status']!={},df['retweeted_status'].apply(lambda row : getFullText(row)),np.where(df['extended_tweet']!={}, df['extended_tweet'].apply(lambda row : getExtTweet(row)),df['text']))
    
    # extract original tweeter's screen name if tweet is retweet
    df.insert(df.columns.get_loc('retweeted_status')+1, 'og_tweet_un', "")
    df['og_tweet_un'] = np.where(df['retweeted_status']!={}, df['retweeted_status'].apply(getOg), "")
    
    # format source column in easily readable way
    df['source'] = df['source'].apply(lambda st: st[st.find(">")+1:st.find("</a")])
    
    # use sentiment analyzer
    df['text'] = df['text'].astype(str)
    df = df.reset_index(drop=True)
    analyzer = SentimentIntensityAnalyzer()
    df['sentiment'] = np.where(df['full_text']=='',[analyzer.polarity_scores(x)['compound'] for x in df['text']],[analyzer.polarity_scores(x)['compound'] for x in df['full_text']])
    
    # get location information from user column
    df.insert(df.columns.get_loc('user')+1, 'location', "")
    df['user'] = df['user'].apply(ast.literal_eval)
    df['location'] = df['user'].apply(lambda row: row['location'])
    
    # get follower count from user column
    df.insert(df.columns.get_loc('user')+1, 'follower_count', "")
    df['follower_count'] = df['user'].apply(lambda row: row['followers_count'])
    
    # get screen name from user column
    df.insert(df.columns.get_loc('user')+1, 'screen_name', "")
    df['screen_name'] = df['user'].apply(lambda row: row['screen_name'])
    
    df = df[final_list]
    df.to_csv(csvPath+pickle.replace('.pkl','')+'.csv')
    all_df.append(df)
    

In [46]:
csvList = os.listdir(csvPath)

In [60]:
all_df = pd.DataFrame(columns=final_list)
for ad in csvList:
    df = pd.read_csv(csvPath+ad)
    all_df = pd.concat([all_df,df])

In [66]:
all_df.to_csv('C:/Users/jacob/Desktop/Data Analytics 2022/2022 Ads/allAds.csv')

In [65]:
all_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2031877 entries, 0 to 396
Data columns (total 19 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   _id                      object 
 1   screen_name              object 
 2   follower_count           object 
 3   location                 object 
 4   Ad Name                  object 
 5   created_at               object 
 6   mediaType                float64
 7   lang                     object 
 8   source                   object 
 9   text                     object 
 10  full_text                object 
 11  og_tweet_un              object 
 12  in_reply_to_screen_name  object 
 13  favorite_count           object 
 14  quote_count              object 
 15  reply_count              object 
 16  retweet_count            object 
 17  sentiment                float64
 18  Unnamed: 0               float64
dtypes: float64(3), object(16)
memory usage: 310.0+ MB


In [22]:
# get formatting correct
for i in range(0,df.columns.get_loc('favorite_count')):
    df.iloc[:,i] = df.iloc[:,i].astype(str)
df = df.replace('nan', np.nan)
df['created_at'] = df['created_at'].astype('datetime64[ns]')

# get entities formatted as json, create mediatype column
df['entities'] = df['entities'].apply(ast.literal_eval)
df.insert(df.columns.get_loc('entities')+1, 'mediaType', np.nan)
df['mediaType'] = df['entities'].apply(lambda cell : medTypeFind(cell))

In [23]:
# format retweeted_status and extended_tweet columns as json, create full_text column
df['retweeted_status'] = df['retweeted_status'].replace(np.nan, "{}")
df['extended_tweet'] = df['extended_tweet'].replace(np.nan, "{}")
df.insert(df.columns.get_loc('text')+1,'full_text',"")
df['retweeted_status'] = df['retweeted_status'].apply(ast.literal_eval)
df['extended_tweet'] = df['extended_tweet'].apply(ast.literal_eval)

In [24]:
# extract full text wherever possible
df['full_text'] = np.where(df['retweeted_status']!={},df['retweeted_status'].apply(lambda row : getFullText(row)),np.where(df['extended_tweet']!={}, df['extended_tweet'].apply(lambda row : getExtTweet(row)),df['text']))

In [25]:
# extract original tweeter's screen name if tweet is retweet
df.insert(df.columns.get_loc('retweeted_status')+1, 'og_tweet_un', "")
df['og_tweet_un'] = np.where(df['retweeted_status']!={}, df['retweeted_status'].apply(getOg), "")

In [26]:
# format source column in easily readable way
df['source'] = df['source'].apply(lambda st: st[st.find(">")+1:st.find("</a")])

In [27]:
# use sentiment analyzer
df['text'] = df['text'].astype(str)
df = df.reset_index(drop=True)
analyzer = SentimentIntensityAnalyzer()
df['sentiment'] = np.where(df['full_text']=='',[analyzer.polarity_scores(x)['compound'] for x in df['text']],[analyzer.polarity_scores(x)['compound'] for x in df['full_text']])

In [28]:
# get location information from user column
df.insert(df.columns.get_loc('user')+1, 'location', "")
df['user'] = df['user'].apply(ast.literal_eval)
df['location'] = df['user'].apply(lambda row: row['location'])

In [29]:
# get follower count from user column
df.insert(df.columns.get_loc('user')+1, 'follower_count', "")
df['follower_count'] = df['user'].apply(lambda row: row['followers_count'])

In [30]:
# get screen name from user column
df.insert(df.columns.get_loc('user')+1, 'screen_name', "")
df['screen_name'] = df['user'].apply(lambda row: row['screen_name'])

In [31]:
df = df[final_list]

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167085 entries, 0 to 167084
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   _id                      167085 non-null  object        
 1   screen_name              167085 non-null  object        
 2   follower_count           167085 non-null  int64         
 3   location                 77266 non-null   object        
 4   Ad Name                  167085 non-null  object        
 5   created_at               167085 non-null  datetime64[ns]
 6   mediaType                0 non-null       float64       
 7   lang                     167085 non-null  object        
 8   source                   167085 non-null  object        
 9   text                     167085 non-null  object        
 10  full_text                167085 non-null  object        
 11  og_tweet_un              167085 non-null  object        
 12  in_reply_to_scre

In [38]:
df.to_csv(csvPath+'amazon.csv')